# ISTM4212_10 Project 2
##### By Group 05: Violet(Yiqi) Chen [Group Lead], Xuening Yang, Yuyang He

>Check Working Directory

In [6]:
!pwd

/home/ubuntu/notebooks/Project2_Group05


In [7]:
#%cd ./Project2_Group05

> Load sql

In [1]:
!pip freeze | grep -E 'ipython-sql|psycopg2'

ipython-sql==0.4.0
psycopg2==2.9.3
psycopg2-binary==2.9.3


In [2]:
%load_ext sql

In [3]:
%sql postgresql://student@/week6

## First Part – ER Data Model Exercise (25 points)

#### <font color='green'>  Explanation about the graph. </font>

Before getting into the first part designing our model, we closely observed the data and try to get some sense out of them. After we identified that some data files have duplicate and some does not have a natural primary key, we decided to use RStudio to process the data. There are 3 things we did: removed some duplicates, generate some primary keys, and alter columns. </br>
+ We removed the duplicate for airports_list, B43_AIRCRAFT_INVENTORY, and CARRIER_DECODE by their primary key.
+ We generate a few primary key for those who do not have a available primary keys or composite primary keys for the easiness to use. There are REPORT_ID for ontime_reporting (not available), S_DATE for airport_weather (composite), UNIQUE_CARRIER_ID for P10_EMPLOYEE (composite).
+ We altered ontime_reporting by adding YEAR. We also altered airport_weather by selecting 7 features that are regarded as important by [raw data documentation](https://www.kaggle.com/threnjen/2019-airline-delays-and-cancellations?select=raw_data_documentation.txt), which are NAME, DATE, PRCP, SNOW, SNWD, TMAX, and AWND as well as STATION selected by us used to generate S_DATE. </br>

Further details about detail steps and R functions as well as packages we used to perform the data cleaning will be available in 2. Constructing your Database – Data Loading of Second Part.

#### <font color='bronze'>1. ER-diagram (10 points)</font>

Refer to DBMS_ER_diagram_ISTM4212.png in the folder

#### <font color='bronze'>2. Relational DB schema (10 points)</font>

Refer to Relational_DB_Schema.xlsx in the folder

#### <font color='bronze'>3. General opinion on the data dictionary files provided. Provide suggestion on improving the design of the data files. (5 points)</font>

In the data preprocessing step, we identified a few issues related to data file provided, which can be improved.<br>
1) **Duplicate**: 
i. In some data dictionary files, there are rows duplicating themselves, especially AIRPORT_COORDINATES, airports_list, and CARRIER_DECODE. They need to be cleaned to make each row unique by primary key. Also, there are duplicate column in the same data file: Entity P10_EMPLOYEES have attribute OP_UNIQUE_CARRIER and UNIQUE_CARRIER_NAME, which contain exact same information as attribute CARRIER and CARRIER_NAME under the same entity. Our suggestion is to keep only either pair given these two pairs are the same.<br>
ii. In some data file, unique key is not unique. In CARRIER_DECODE.csv, according to the data description - "Display carrier name by the airline id or carrier code" - we expect both airline and carrier code to be unique key. However, doing unique by airline id or by carrier code returns different number of rows (1686 obs. and 1744 obs. respectively). On top of that, it is reasonable that a carrier may have multiple carrier codes, but a carrier code should not be shared by multiple carrier. However, for example, OP_UNIQUE_CARRIER "4E" which belongs to British Airtours Limited is also displayed as Tanana Air Service's carrier code whose code is actually "Z3" according to our research (refers to [Bureau of Transportation Statistics](https://www.bts.gov/topics/airlines-and-airports/airline-codes)). We suspect there may be some error in data collecting process, we suggest this data file to be regenerated.
2) **Inconsistency**: We identified inconsistency in the data file, in both label/field/column names and the format of data.<br>
i. One example of different column names will be airport ID, which appear as AIRLINE_ID, ORIGIN_AIRPORT_ID. Since both AIRLINE_ID and AIRPORT_ID refers to the same thing, for the easiness of understanding and database operation, we suggest to use same attribute name (either AIRLINE_ID or AIRPORT_ID) for most entities, except for ONTIME_REPORTING which has the need to differetiate between origin and destination.<br>
ii. In the AIRPORT_WEATHER entity, the attribute DATE is formatted as both mm/dd/yyyy and yyyy-mm-dd in data file airport_weather_2019.csv but as yyyy-mm-dd in airport_weather_2020.csv. To avoid error when convert to DATE data type, we suggest to format this attribute consistently as yyyy-mm-dd.<br>
iii. Also, in the AIRPORT_WEATHER entity, the columes in 2020 and 2019 csv files are ordered different, which will create mismatch when using csvstack. Therefore, we suggest that the columns should be arranged in same order and be consistent with same amount of columns (2020 has column WT18 while 2019 does not; 2019 has columns TOBS and WT11 while 2020 does not).
3) **Difficulty of Locating Primary Key**: For some data, they are unique each row, but do not have primary keys which are easy to use. We suggest that primary key can be generate using simply index or meaningful infomation from multiple attributes to make it unique.

## Second Part – Building a Physical Data model and construct the database (75 points)

#### <font color='bronze'>1. Creating your Database Objects (20 points)</font>

##### <font color='grey'>Using the schema diagram from first part above, create the physical tables for all entities and their relationships. Make sure the tables contain all primary, foreign, and unique keys as well as any default, and check constraints (if any). The tables must also Indicate the null constraint for all attributes. (All entities from first part are required)</font>

+ Create Table WEATHER
<br/> We generate our own primary key S_DATE based on STATION and DATE. We only use 9 variable according to the raw_data_documentation.txt which says that Important features are NAME, DATE, PRCP, SNOW, SNWD, TMAX, and AWND. Further explanation will be available in the section where we copy the csv file into the table.

In [15]:
%%sql
DROP TABLE IF EXISTS WEATHER Cascade;

CREATE TABLE WEATHER (
    S_DATE varchar(50) Not NULL, 
    STATION varchar(50) Not NULL,
    NAME varchar(108) NOT NULL,
    DATE DATE NOT NULL,
    PRCP Numeric(4,2) NULL,
    SNOW Numeric(4,2) NULL,
    SNWD Numeric(4,2) NULL,
    TMAX Numeric(5,2) NULL,
    AWND Numeric(4,2) NULL,
    PRIMARY KEY (S_DATE),
    UNIQUE (STATION, DATE),
    UNIQUE (NAME, DATE)
);

COMMENT ON TABLE WEATHER is 'Airport weather by date';
COMMENT ON COLUMN WEATHER.S_DATE is 'Generated primary key based on STATION and DATE';
COMMENT ON COLUMN WEATHER.STATION is 'Location code';
COMMENT ON COLUMN WEATHER.NAME is 'Location of reading';
COMMENT ON COLUMN WEATHER.DATE is 'Date of reading';

 * postgresql://student@/week6
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [16]:
%%sql
select * from weather

 * postgresql://student@/week6
0 rows affected.


s_date,station,name,date,prcp,snow,snwd,tmax,awnd


+ Create Table AIRPORT_COORDINATES

In [18]:
%%sql
DROP TABLE IF EXISTS AIRPORT_COORDINATES Cascade;

CREATE TABLE AIRPORT_COORDINATES (
  ORIGIN_AIRPORT_ID   Numeric(5) Not NULL, 
  DISPLAY_AIRPORT_NAME   varchar(50) NOT NULL,
  LATITUDE Numeric(10,7) NULL,
  LONGITUDE  Numeric(10,7) NULL, 
  PRIMARY KEY (ORIGIN_AIRPORT_ID),
  UNIQUE (DISPLAY_AIRPORT_NAME)
);

COMMENT ON TABLE AIRPORT_COORDINATES is
'AIRPORT_COORDINATES: Latitude and Longitude coordinates of airports, by airport name and airport id';
COMMENT ON COLUMN AIRPORT_COORDINATES.ORIGIN_AIRPORT_ID is 'Airport ID, matches to ORIGIN_AIRPORT_ID in other files';
COMMENT ON COLUMN AIRPORT_COORDINATES.DISPLAY_AIRPORT_NAME is 'Display Airport, matches to DISPLAY_AIRPORT_NAME in other files';

 * postgresql://student@/week6
Done.
Done.
Done.
Done.
Done.


[]

In [19]:
%%sql
select * from AIRPORT_COORDINATES

 * postgresql://student@/week6
0 rows affected.


origin_airport_id,display_airport_name,latitude,longitude


+ Create Table AIRPORT_LIST

In [20]:
%%sql
DROP TABLE IF EXISTS AIRPORT_LIST Cascade;

CREATE TABLE AIRPORT_LIST (
  ORIGIN_AIRPORT_ID   Numeric(5) Not NULL, 
  DISPLAY_AIRPORT_NAME   varchar(108) NOT NULL,
  ORIGIN_CITY_NAME varchar(50) NULL,
  NAME  varchar(108) NULL, 
  PRIMARY KEY (ORIGIN_AIRPORT_ID),
  UNIQUE (DISPLAY_AIRPORT_NAME)
);

COMMENT ON TABLE AIRPORT_LIST is
'AIRPORT_COORDINATES: Airport display names; used with weather files';
COMMENT ON COLUMN AIRPORT_LIST.ORIGIN_AIRPORT_ID is 'Airport ID, matches to ORIGIN_AIRPORT_ID in other files';
COMMENT ON COLUMN AIRPORT_LIST.DISPLAY_AIRPORT_NAME is 'Display Airport, matches to DISPLAY_AIRPORT_NAME in other files';
COMMENT ON COLUMN AIRPORT_LIST.NAME is 'Matches to NAME in airport_weather';

 * postgresql://student@/week6
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [21]:
%%sql
select * from airport_list

 * postgresql://student@/week6
0 rows affected.


origin_airport_id,display_airport_name,origin_city_name,name


+ Create Table AIRCRAFT

In [22]:
%%sql
DROP TABLE IF EXISTS AIRCRAFT Cascade;

CREATE TABLE AIRCRAFT (
  MANUFACTURE_YEAR Numeric(4) NULL, 
  TAIL_NUM   varchar(50) NOT NULL,
  NUMBER_OF_SEATS Numeric(3) NULL,
  PRIMARY KEY (TAIL_NUM)
);

COMMENT ON TABLE AIRCRAFT is
'AIRCRAFT: Manufacture year and passenger capacity for aircraft by tail number';
COMMENT ON COLUMN AIRCRAFT.TAIL_NUM is 'Unique tail number, matches to TAIL_NUM in other files';

 * postgresql://student@/week6
Done.
Done.
Done.
Done.


[]

In [23]:
%%sql
select * from aircraft

 * postgresql://student@/week6
0 rows affected.


manufacture_year,tail_num,number_of_seats


+ Create Table CARRIER
<br/> may need to redo data cleaning, do unique to carrier name

In [24]:
%%sql
DROP TABLE IF EXISTS CARRIER Cascade;

CREATE TABLE CARRIER (
  AIRLINE_ID Numeric(5) NULL, --ATTENTATIVE
  OP_UNIQUE_CARRIER varchar(50) NOT NULL,
  CARRIER_NAME varchar(108) NULL,
  PRIMARY KEY (OP_UNIQUE_CARRIER)
 );

COMMENT ON TABLE CARRIER is 'CARRIER: Display carrier name by the airline id or carrier code';
COMMENT ON COLUMN CARRIER.AIRLINE_ID is 'Airport ID, matches to AIRLINE_ID in other files';
COMMENT ON COLUMN CARRIER.OP_UNIQUE_CARRIER is 'Carrier code, matches to OP_UNIQUE_CARRIER in other files';
COMMENT ON COLUMN CARRIER.CARRIER_NAME is 'Carrier name, matches to UNIQUE_CARRIER_NAME or CARRIER_NAME in other files';

 * postgresql://student@/week6
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [25]:
%%sql
select * from carrier

 * postgresql://student@/week6
0 rows affected.


airline_id,op_unique_carrier,carrier_name


+ Create Table P10_EMPLOYEES </br>
We recognized that this entity has a composite primary key -- OP_UNIQUE_CARRIER and ENTITY. For the easiness of use, UNIQUE_CARRIER_ID is a generated primary key based on OP_UNIQUE_CARRIER and ENTITY

In [27]:
%%sql
DROP TABLE IF EXISTS P10_EMPLOYEES Cascade;

CREATE TABLE P10_EMPLOYEES (
    UNIQUE_CARRIER_ID varchar(50) NOT NULL,
    YEAR Numeric(4)NOT NULL,
    AIRLINE_ID Numeric(5) NOT NULL,
    OP_UNIQUE_CARRIER varchar(50) NOT NULL,
    UNIQUE_CARRIER_NAME varchar(108) NOT NULL,
    ENTITY char(1) NOT NULL,
    GENERAL_MANAGE Numeric(4) NULL,
    PILOTS_COPILOTS Numeric(4) NULL,
    OTHER_FLT_PERS Numeric(4) NULL,
    PASS_GEN_SVC_ADMIN Numeric(5) NULL,
    MAINTENANCE Numeric(5) NULL,
    ARCFT_TRAF_HANDLING_GRP1 Numeric(4) NULL,
    GEN_ARCFT_TRAF_HANDLING Numeric(4) NULL,
    AIRCRAFT_CONTROL Numeric(4) NULL,
    PASSENGER_HANDLING Numeric(5) NULL,
    CARGO_HANDLING Numeric(6) NULL,
    TRAINEES_INTRUCTOR Numeric(4) NULL,
    STATISTICAL Numeric(5) NULL,
    TRAFFIC_SOLICITERS Numeric(4) NULL,
    OTHER Numeric(5) NULL,
    TRANSPORT_RELATED Numeric(5) NULL,
    TOTAL Numeric(5) NULL,
    PRIMARY KEY (UNIQUE_CARRIER_ID),
    UNIQUE (OP_UNIQUE_CARRIER, ENTITY)
    -- FOREIGN KEY (OP_UNIQUE_CARRIER) references CARRIER(OP_UNIQUE_CARRIER)
);

COMMENT ON TABLE P10_EMPLOYEES is 'P10_EMPLOYEES: Employees per carrier';
COMMENT ON COLUMN P10_EMPLOYEES.AIRLINE_ID is 'Airport ID, matches to AIRLINE_ID in other files';
COMMENT ON COLUMN P10_EMPLOYEES.OP_UNIQUE_CARRIER is 'Carrier code, matches to OP_UNIQUE_CARRIER in other files';
COMMENT ON COLUMN P10_EMPLOYEES.UNIQUE_CARRIER_NAME is 'Carrier name, matches to UNIQUE_CARRIER_NAME or CARRIER_NAME in other files';

 * postgresql://student@/week6
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [28]:
%%sql
select * from P10_EMPLOYEES

 * postgresql://student@/week6
0 rows affected.


unique_carrier_id,year,airline_id,op_unique_carrier,unique_carrier_name,entity,general_manage,pilots_copilots,other_flt_pers,pass_gen_svc_admin,maintenance,arcft_traf_handling_grp1,gen_arcft_traf_handling,aircraft_control,passenger_handling,cargo_handling,trainees_intructor,statistical,traffic_soliciters,other,transport_related,total


+ Create Table ONTIME_REPORTING </br>
Compared to index, we rather use a more meaningful derived primary key. REPORT_ID is a generated primary key using OP_UNIQUE_CARRIER, OP_CARRIER_FL_NUM, ORIGIN_AIRPORT_ID, YEAR, MONTH, DAY_OF_MONTH, TAIL_NUM based on the assumption that a **aircraft** of a certain **carrier** can only engage in a certain **air route** departing from certain **airport** in a certain **day**.

In [29]:
%%sql
DROP TABLE IF EXISTS ONTIME_REPORTING Cascade;

CREATE TABLE ONTIME_REPORTING (
  REPORT_ID   varchar(50) Not NULL, 
  YEAR   numeric(4) NOT NULL,
  MONTH  numeric(1) NOT NULL,
  DAY_OF_MONTH   numeric(2) NOT NULL,
  DAY_OF_WEEK    numeric(1)  NOT NULL,   
  OP_UNIQUE_CARRIER varchar(10) NOT NULL,
  TAIL_NUM  varchar(10) NULL,
  OP_CARRIER_FL_NUM numeric(4) NOT NULL,
  ORIGIN_AIRPORT_ID numeric(5) NOT NULL,
  ORIGIN  CHARACTER(10) NOT NULL,
  ORIGIN_CITY_NAME varchar(50) NOT NULL,
  DEST_AIRPORT_ID numeric(5) NOT NULL,
  DEST  varchar(15) NOT NULL,
  DEST_CITY_NAME  varchar(50) NOT NULL,
  CRS_DEP_TIME numeric(4) NOT NULL,
  DEP_TIME  numeric(4) NULL,
  DEP_DELAY_NEW  numeric(6) NULL,
  DEP_DEL15 numeric(2) NULL,
  DEP_TIME_BLK VARCHAR(10) NOT NULL,
  CRS_ARR_TIME  numeric(5) NOT NULL,
  ARR_TIME   numeric(4) NULL,
  ARR_DELAY_NEW  numeric(4) NULL,
  ARR_TIME_BLK VARCHAR(10) NOT NULL,
  CANCELLED  numeric(2) NULL,
  CANCELLATION_CODE VARCHAR(1) NULL,
  CRS_ELAPSED_TIME numeric(4) NULL,
  ACTUAL_ELAPSED_TIME numeric(4) NULL,
  DISTANCE  numeric(4) NOT NULL,
  DISTANCE_GROUP numeric(2) NOT NULL,
  CARRIER_DELAY  numeric(4) NULL,
  WEATHER_DELAY  numeric(4) NULL,
  NAS_DELAY  numeric(4) NULL,
  SECURITY_DELAY numeric(4) NULL,
  LATE_AIRCRAFT_DELAY numeric(4) NULL,
  X varchar(2), -- what is this?
  PRIMARY KEY (REPORT_ID),
  FOREIGN KEY (OP_UNIQUE_CARRIER) references CARRIER(OP_UNIQUE_CARRIER)
-- The foreign key in comments cannot connect because the data in the other table is not complete.
-- Futher explanation will be available in the bottom of Part 2 question 2.
  -- FOREIGN KEY (TAIL_NUM) references AIRCRAFT(TAIL_NUM),
  -- FOREIGN KEY (ORIGIN_AIRPORT_ID) references AIRPORT_COORDINATES(ORIGIN_AIRPORT_ID) ,
  -- FOREIGN KEY (DEST_AIRPORT_ID) references AIRPORT_COORDINATES(ORIGIN_AIRPORT_ID)
  -- FOREIGN KEY (ORIGIN_AIRPORT_ID) references AIRPORT_LIST(ORIGIN_AIRPORT_ID) ,
  -- FOREIGN KEY (DEST_AIRPORT_ID) references AIRPORT_LIST(ORIGIN_AIRPORT_ID)
-- UNIQUE (CNAME)
);

 * postgresql://student@/week6
Done.
Done.


[]

In [30]:
%%sql
select * from ONTIME_REPORTING

 * postgresql://student@/week6
0 rows affected.


report_id,year,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay_new,dep_del15,dep_time_blk,crs_arr_time,arr_time,arr_delay_new,arr_time_blk,cancelled,cancellation_code,crs_elapsed_time,actual_elapsed_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,x


#### <font color='bronze'>2. Constructing your Database – Data Loading (10 points)</font>

##### <font color='grey'> Once you have created the tables and their constraints, construct (i.e., populate) the tables in bulk using the data files for year 2019 (Jan-Mar) and 2020 (Jan-Mar). Make sure to check your answer (using select queries) against the original text files (using Linux or csvkit commands) </font>

%%sql 
-- checking what table do we have
SELECT *
FROM pg_catalog.pg_tables where tableowner='student'

+ Import AIRPORT_COORDINATES

Data cleaning using **RStudio**: </br>

We remove the duplicate for our selected primary key using </br>
> df <- df[!duplicated(df$origin_airport_id),]

In [33]:
!wc -l AIRPORT_COORDINATES.csv

6487 AIRPORT_COORDINATES.csv


In [34]:
%%sql
COPY AIRPORT_COORDINATES FROM '/home/ubuntu/notebooks/Project2_Group05/AIRPORT_COORDINATES.csv'
CSV
HEADER;

 * postgresql://student@/week6
6486 rows affected.


[]

We used Linux to check the rows affected then we got 6487 rows which is one more than 6486 rows in the original dataset, AIRPORT_COORDINATES.csv. This is because we excluded the header when we inspect the data, therefore we used the same data in the next constructing table step.

In [38]:
!head -3 AIRPORT_COORDINATES.csv

"ORIGIN_AIRPORT_ID","DISPLAY_AIRPORT_NAME","LATITUDE","LONGITUDE"
10001,"Afognak Lake Airport",58.10944444,-152.9066667
10003,"Bear Creek Mining Strip",65.54805556,-161.0716667


In [35]:
%%sql
select * from AIRPORT_COORDINATES limit 2

 * postgresql://student@/week6
2 rows affected.


origin_airport_id,display_airport_name,latitude,longitude
10001,Afognak Lake Airport,58.1094444,-152.9066667
10003,Bear Creek Mining Strip,65.5480556,-161.0716667


+ Import weather </br>

Data cleaning using **RStudio**: </br>

The column names and column number of 2019 and 2020 csv file is different, which hindered us from directly using csvstack, To solve this problem, we used RStudio to select only the useful attributes and arrange them in certain order.According to the raw_data_documentation.txt which says that Important features are NAME, DATE, PRCP, SNOW, SNWD, TMAX, and AWND, we only include these attributes and STATION as well as our generated attribute S_DATE. We select using function
> df <-  select(df,c("STATION", "NAME", "DATE", "PRCP", "SNOW, SNWD, TMAX, and AWND))

We observed that there are 2 formats of date presented in Date. Thus, we convert it to the standard DATE data type used in SQL by
> library(lubridate) </br>
mdy <- mdy(df$DATE) </br>
dmy <- ymd(df$DATE) </br>
mdy[is.na(mdy)] <- dmy[is.na(mdy)] </br>
df$DATE<- mdy </br>

As we are awared of the composite unique key (station, date) or unique key (name, date), we decide to generate a primary key based on STATION and DATE </br>
> df <- mutate(df,S_DATE = paste0(STATION, DATE), .before = "STATION")

and remove the slash using </br>
> df$S_DATE <- gsub("-","",df$S_DATE)

We cut out the data from January to March for 2019 weather data using
> df <- df[df$DATE >= "2019-01-01" & df$DATE <= "2019-03-31", ]

In [5]:
!csvstack airport_weather_20*.csv > airport_weather.csv

In [36]:
!wc -l airport_weather.csv

17084 airport_weather.csv


In [40]:
%%sql
COPY WEATHER FROM '/home/ubuntu/notebooks/Project2_Group05/airport_weather.csv'
CSV
HEADER;

 * postgresql://student@/week6
17083 rows affected.


[]

We used Linux to check the rows affected then we got 17084 rows which is one more than 17083 rows in the original dataset, airport_weather.csv. This is because we excluded the header when we inspect the data, therefore we used the same data in the next constructing table step.

In [39]:
!head -3 airport_weather.csv

S_DATE,STATION,NAME,DATE,PRCP,SNOW,SNWD,TMAX,AWND
USW0001387420190101,USW00013874,"ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US",2019-01-01,0.14,0,0,66,4.7
USW0001387420190102,USW00013874,"ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US",2019-01-02,0.57,0,0,59,4.92


In [41]:
%%sql
select * from WEATHER limit 2

 * postgresql://student@/week6
2 rows affected.


s_date,station,name,date,prcp,snow,snwd,tmax,awnd
USW0001387420190101,USW00013874,"ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US",2019-01-01,0.14,0.00,0.00,66.00,4.70
USW0001387420190102,USW00013874,"ATLANTA HARTSFIELD JACKSON INTERNATIONAL AIRPORT, GA US",2019-01-02,0.57,0.00,0.00,59.00,4.92


+ Import AIRPORT_LIST

Data cleaning: </br>
We did not do cleaning to this file because each row is uniquely identified by ORIGIN_AIRPORT_ID.

In [42]:
!wc -l airports_list.csv

98 airports_list.csv


In [43]:
%%sql
COPY AIRPORT_LIST FROM '/home/ubuntu/notebooks/Project2_Group05/airports_list.csv'
CSV
HEADER;

 * postgresql://student@/week6
97 rows affected.


[]

We used Linux to check the rows affected then we got 98 rows which is one more than 97 rows in the original dataset, airport_list.csv. This is because we excluded the header when we inspect the data, therefore we used the same data in the next constructing table step.

In [44]:
!head -3 airports_list.csv

ORIGIN_AIRPORT_ID,DISPLAY_AIRPORT_NAME,ORIGIN_CITY_NAME,NAME
12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US"
10257,Albany International,"Albany, NY","ALBANY INTERNATIONAL AIRPORT, NY US"


In [16]:
%%sql
select * from AIRPORT_LIST limit 2

 * postgresql://student@/week6
3 rows affected.


origin_airport_id,display_airport_name,origin_city_name,name
12992,Adams Field,"Little Rock, AR","NORTH LITTLE ROCK AIRPORT, AR US"
10257,Albany International,"Albany, NY","ALBANY INTERNATIONAL AIRPORT, NY US"
10140,Albuquerque International Sunport,"Albuquerque, NM","ALBUQUERQUE INTERNATIONAL AIRPORT, NM US"


+ Import AIRCRAFT

Data cleaning using **RStudio**: </br>
We check the duplicate based on selected primary key TAIL_NUM by
> df[duplicated(df$TAIL_NUM),]

We notice that some duplicates have complete records while the others do not. Thus, we create a table with duplicated record using
> dup <- df[duplicated(df$TAIL_NUM),] </br>
plyr::match_df(df,dup, on = 'TAIL_NUM')

and then careful remove duplicates while preserving those with non-zero NUMBER_OF_SEATS. Then, we get rid of all duplicate of the same records by 
> df <- df[!duplicated(df$TAIL_NUM),]

Besides, there is a cross in around the six-thousandth line in the data file causing error saying "the file is not UTF-8 encoded." We simply find the cross and delete it to solve the error.

In [46]:
!wc -l B43_AIRCRAFT_INVENTORY.csv

7362 B43_AIRCRAFT_INVENTORY.csv


We used Linux to check the rows affected then we got 7362 rows which is one more than 7361 rows in the original dataset, B43_AIRCRAFT_INVENTORY.csv. This is because we excluded the header when we inspect the data, therefore we used the same data in the next constructing table step.

In [47]:
%%sql
COPY AIRCRAFT FROM '/home/ubuntu/notebooks/Project2_Group05/B43_AIRCRAFT_INVENTORY.csv'
CSV
HEADER;

 * postgresql://student@/week6
7361 rows affected.


[]

In [45]:
!head -3 B43_AIRCRAFT_INVENTORY.csv

"MANUFACTURE_YEAR","TAIL_NUM","NUMBER_OF_SEATS"
1944,"N54514",0
1945,"N1651M",0


In [48]:
%%sql
select * from AIRCRAFT limit 2

 * postgresql://student@/week6
2 rows affected.


manufacture_year,tail_num,number_of_seats
1944,N54514,0
1945,N1651M,0


+ Import P10_EMPLOYEES

Data cleaning using **RStudio**: </br>
We are awared of the composite primary key (OP_UNIQUE_CARRIER, ENTITY). But for the easiness of use, we decided to generate a primary key based on OP_UNIQUE_CARRIER and ENTITY
> df <- mutate(df,UNIQUE_CARRIER_ID = paste0(OP_UNIQUE_CARRIER, ENTITY), .before = "YEAR")

In [50]:
!wc -l P10_EMPLOYEES.csv

90 P10_EMPLOYEES.csv


In [51]:
%%sql
COPY P10_EMPLOYEES FROM '/home/ubuntu/notebooks/Project2_Group05/P10_EMPLOYEES.csv'
CSV
HEADER;

 * postgresql://student@/week6
89 rows affected.


[]

We used Linux to check the rows affected then we got 90 rows which is one more than 89 rows in the original dataset, P10_EMPLOYEES.csv This is because we excluded the header when we inspect the data, therefore we used the same data in the next constructing table step.

In [52]:
!head -3 P10_EMPLOYEES.csv

"UNIQUE_CARRIER_ID","YEAR","AIRLINE_ID","OP_UNIQUE_CARRIER","UNIQUE_CARRIER_NAME","ENTITY","GENERAL_MANAGE","PILOTS_COPILOTS","OTHER_FLT_PERS","PASS_GEN_SVC_ADMIN","MAINTENANCE","ARCFT_TRAF_HANDLING_GRP1","GEN_ARCFT_TRAF_HANDLING","AIRCRAFT_CONTROL","PASSENGER_HANDLING","CARGO_HANDLING","TRAINEES_INTRUCTOR","STATISTICAL","TRAFFIC_SOLICITERS","OTHER","TRANSPORT_RELATED","TOTAL"
"0WQD",2019,21352,"0WQ","Avjet Corporation","D",4,53,6,19,29,21,0,0,0,3,1,18,0,7,0,161
"1BQI",2019,21492,"1BQ","Eastern Airlines f/k/a Dynamic Airways, LLC","I",14,50,0,41,18,21,0,0,0,0,1,13,0,3,0,161


In [53]:
%%sql
select * from P10_EMPLOYEES limit 2

 * postgresql://student@/week6
2 rows affected.


unique_carrier_id,year,airline_id,op_unique_carrier,unique_carrier_name,entity,general_manage,pilots_copilots,other_flt_pers,pass_gen_svc_admin,maintenance,arcft_traf_handling_grp1,gen_arcft_traf_handling,aircraft_control,passenger_handling,cargo_handling,trainees_intructor,statistical,traffic_soliciters,other,transport_related,total
0WQD,2019,21352,0WQ,Avjet Corporation,D,4,53,6,19,29,21,0,0,0,3,1,18,0,7,0,161
1BQI,2019,21492,1BQ,"Eastern Airlines f/k/a Dynamic Airways, LLC",I,14,50,0,41,18,21,0,0,0,0,1,13,0,3,0,161


+ Import CARRIER

Data cleaning using **RStudio**: </br>

We get rid of duplicate based of selected primary key OP_UNIQUE_CARRIER using
> df <- df[!duplicated(df$OP_UNIQUE_CARRIER),]

We notice that while some records have same AIRLINE_ID and CARRIER_NAME, their OP_UNIQUE_CARRIER could be different. We find it curious. However, out of prudence, we would rather not delete these records until further investigation on carrier decode is done.

In [54]:
!wc -l CARRIER_DECODE.csv

1745 CARRIER_DECODE.csv


In [55]:
%%sql
COPY CARRIER FROM '/home/ubuntu/notebooks/Project2_Group05/CARRIER_DECODE.csv'
CSV
HEADER;

 * postgresql://student@/week6
1744 rows affected.


[]

We used Linux to check the rows affected then we got 1745 rows which is one more than 1744 rows in the original dataset, CARRIER_DECODE.csv. This is because we excluded the header when we inspect the data, therefore we used the same data in the next constructing table step.

In [56]:
!head -3 CARRIER_DECODE.csv

"AIRLINE_ID","OP_UNIQUE_CARRIER","CARRIER_NAME"
21754,"2PQ","21 Air LLC"
20342,"Q5","40-Mile Air"


In [57]:
%%sql
select * from CARRIER limit 2

 * postgresql://student@/week6
2 rows affected.


airline_id,op_unique_carrier,carrier_name
21754,2PQ,21 Air LLC
20342,Q5,40-Mile Air


+ Import ONTIME_REPORTING

Data cleaning using **RStudio**: </br>
We noticed that this dataset does not have a natural primary key. Instead of using index, we decided to generate one that allows us to uniquely identify each record. Firstly, we add a column YEAR to the files. </br>
>library(dplyr)
df <- mutate(df,YEAR = 2019, .before = "MONTH") for ontime reporting of 2019
df <- mutate(df,YEAR = 2020, .before = "MONTH") for ontime reporting of 2019

Then, we generated primary key using OP_UNIQUE_CARRIER, OP_CARRIER_FL_NUM, ORIGIN_AIRPORT_ID, YEAR, MONTH, DAY_OF_MONTH, TAIL_NUM based on the assumption that a **aircraft** of a certain **carrier** can only engage in a certain **air route** departing from certain **airport** in a certain **day**. </br>
> df <- mutate(df,REPORT_ID = paste0(OP_UNIQUE_CARRIER, OP_CARRIER_FL_NUM, ORIGIN_AIRPORT_ID, YEAR, MONTH, DAY_OF_MONTH, TAIL_NUM), .before = "YEAR")

which can uniquely identify each row, because it return 0 row when we test our assumption with 
> df[duplicated(df$REPORT_ID),]

In [15]:
!wc -l c_ONTIME_REPORTING_20*.csv

   583986 c_ONTIME_REPORTING_2019_01.csv
   533176 c_ONTIME_REPORTING_2019_02.csv
   632075 c_ONTIME_REPORTING_2019_03.csv
   607347 c_ONTIME_REPORTING_2020_01.csv
   574269 c_ONTIME_REPORTING_2020_02.csv
   648230 c_ONTIME_REPORTING_2020_03.csv
  3579083 total


In [16]:
#!csvcut -n c_ONTIME_REPORTING_2019_01.csv

In [16]:
!csvstack c_ONTIME_REPORTING_20*.csv > ONTIME_REPORTING_JANtoMAR.csv

In [58]:
!wc -l ONTIME_REPORTING_JANtoMAR.csv

3579078 ONTIME_REPORTING_JANtoMAR.csv


In [59]:
%%sql
COPY ONTIME_REPORTING FROM '/home/ubuntu/notebooks/Project2_Group05/ONTIME_REPORTING_JANtoMAR.csv'
CSV
HEADER;

 * postgresql://student@/week6
3579077 rows affected.


[]

We merged ontime reporting 2019 and 2020 together, 3579083 total, to a new file, ONTIME_REPORTING_JANtoMAR.csv. We used Linux to check the rows affected then we got 3579078 rows (5 rows missing from "c_ONTIME_REPORTING_20*.csv" because of 5 headers) which is one more than 3579077 rows in the previous dataset, ONTIME_REPORTING_JANtoMAR.csv. This is because we excluded the header when we inspect the data, therefore we used the same data in the next constructing table step.

In [60]:
!head -3 ONTIME_REPORTING_JANtoMAR.csv

REPORT_ID,YEAR,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,CRS_DEP_TIME,DEP_TIME,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,CRS_ARR_TIME,ARR_TIME,ARR_DELAY_NEW,ARR_TIME_BLK,CANCELLED,CANCELLATION_CODE,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,DISTANCE,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,X
9E328010397201916N8694A,2019,1,6,7,9E,N8694A,3280,10397,ATL,"Atlanta, GA",11150,CSG,"Columbus, GA",1645,1643,0,0,1600-1659,1732,1720,0,1700-1759,0,,47,37,83,1,,,,,,
9E328010397201917N8970D,2019,1,7,1,9E,N8970D,3280,10397,ATL,"Atlanta, GA",11150,CSG,"Columbus, GA",1645,1640,0,0,1600-1659,1732,1712,0,1700-1759,0,,47,32,83,1,,,,,,


In [61]:
%%sql
select * from ONTIME_REPORTING limit 2

 * postgresql://student@/week6
2 rows affected.


report_id,year,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay_new,dep_del15,dep_time_blk,crs_arr_time,arr_time,arr_delay_new,arr_time_blk,cancelled,cancellation_code,crs_elapsed_time,actual_elapsed_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,x
9E328010397201916N8694A,2019,1,6,7,9E,N8694A,3280,10397,ATL,"Atlanta, GA",11150,CSG,"Columbus, GA",1645,1643,0,0,1600-1659,1732,1720,0,1700-1759,0,None,47,37,83,1,None,None,None,None,None,None
9E328010397201917N8970D,2019,1,7,1,9E,N8970D,3280,10397,ATL,"Atlanta, GA",11150,CSG,"Columbus, GA",1645,1640,0,0,1600-1659,1732,1712,0,1700-1759,0,None,47,32,83,1,None,None,None,None,None,None


#### <font color='green'>  Further explanation about foreign key. </font>

In [6]:
%%sql
With diff as (
    Select origin_airport_id from ONTIME_REPORTING
    EXCEPT select origin_airport_id from AIRPORT_COORDINATES)
select count(*) from diff
-- There are 4 origin_airport_id present in ONTIME_REPORTING but not in AIRPORT_COORDINATES. 
    -- Therefore we cannot use foreign key to link this two tables.

 * postgresql://student@/week6
1 rows affected.


count
4


In [7]:
%%sql
With diff as (
    Select dest_airport_id from ONTIME_REPORTING
    EXCEPT select origin_airport_id from AIRPORT_COORDINATES)
select count(*) from diff
-- There are 4 dest_airport_id present in ONTIME_REPORTING but not in AIRPORT_COORDINATES. 
    -- Therefore we cannot use foreign key to link this two tables.

 * postgresql://student@/week6
1 rows affected.


count
4


In [5]:
%%sql
With diff as (
    Select tail_num from ONTIME_REPORTING
    EXCEPT select tail_num from AIRCRAFT)
select count(*) from diff
-- There are 383 tail_num present in ONTIME_REPORTING but not in AIRCRAFT. 
    -- Therefore we cannot use foreign key to link this two tables.

 * postgresql://student@/week6
1 rows affected.


count
383


In [8]:
%%sql
With diff as (
    Select op_unique_carrier from ONTIME_REPORTING
    EXCEPT select op_unique_carrier from CARRIER)
select count(*) from diff
-- There are 0 op_unique_carrier present in ONTIME_REPORTING but not in CARRIER. 
    -- Therefore we can use foreign key to link this two tables.

 * postgresql://student@/week6
1 rows affected.


count
0


In [11]:
%%sql
With diff as (
    Select op_unique_carrier from P10_EMPLOYEES
    EXCEPT select op_unique_carrier from CARRIER)
select count(*) from diff
-- There are 0 op_unique_carrier present in ONTIME_REPORTING but not in CARRIER. 
    -- Therefore we cannot use foreign key to link this two tables.

 * postgresql://student@/week6
1 rows affected.


count
0


In [12]:
%%sql
With diff as (
    Select origin_airport_id from ONTIME_REPORTING
    EXCEPT select origin_airport_id from AIRPORT_LIST)
select count(*) from diff
-- There are 258 origin_airport_id present in ONTIME_REPORTING but not in AIRPORT_LIST.
    -- Therefore we cannot use foreign key to link this two tables.

 * postgresql://student@/week6
1 rows affected.


count
258


In [13]:
%%sql
With diff as (
    Select dest_airport_id from ONTIME_REPORTING
    EXCEPT select origin_airport_id from AIRPORT_LIST)
select count(*) from diff
-- There are 258 dest_airport_id present in ONTIME_REPORTING but not in AIRPORT_LIST. 
    -- Therefore we cannot use foreign key to link this two tables.

 * postgresql://student@/week6
1 rows affected.


count
258


In [14]:
%%sql
With diff as (
    Select name from AIRPORT_LIST
    EXCEPT select name from WEATHER)
select count(*) from diff
-- There are 1 name present in WEATHER but not in AIRPORT_LIST. 
    -- Therefore we cannot use foreign key to link this two tables. 

 * postgresql://student@/week6
1 rows affected.


count
1


#### <font color='bronze'>3. Change Management (10 points)</font>

##### <font color='grey'> Your sponsor asked you to manipulate the original flight date. Write a query that combine day, month, and year of the on-time flight report file into one field. Name the new field “Flight_date”. Once created and checked you can drop the original three fields. For date conversion check https://www.postgresql.org/docs/12/functions-formatting.html.

Create Test_ONTIME_REPORTING and select few data to test the methodology

In [6]:
%%sql
DROP TABLE if exists Test_ONTIME_REPORTING Cascade;

create table Test_ONTIME_REPORTING as
SELECT * from ONTIME_REPORTING
WHERE op_unique_carrier = '9E' and day_of_week = 1;

SELECT * from Test_ONTIME_REPORTING LIMIT 2;

 * postgresql://student@/week6
Done.
20176 rows affected.
2 rows affected.


report_id,year,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay_new,dep_del15,dep_time_blk,crs_arr_time,arr_time,arr_delay_new,arr_time_blk,cancelled,cancellation_code,crs_elapsed_time,actual_elapsed_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,x
9E5452103972019325N8974C,2019,3,25,1,9E,N8974C,5452,10397,ATL,"Atlanta, GA",12951,LFT,"Lafayette, LA",1002,955,0,0,1000-1059,1101,1043,0,1100-1159,0,None,119,108,503,3,None,None,None,None,None,None
9E5451107212019318N915XJ,2019,3,18,1,9E,N915XJ,5451,10721,BOS,"Boston, MA",14492,RDU,"Raleigh/Durham, NC",1422,1415,0,0,1400-1459,1637,1634,0,1600-1659,0,None,135,139,612,3,None,None,None,None,None,None


In [7]:
%%sql
-- here we add a column called Flight_date in our test table
ALTER table Test_ONTIME_REPORTING 
ADD COLUMN Flight_date DATE NULL;

 * postgresql://student@/week6
Done.


[]

In [25]:
%%sql
-- here we test how can we concatenate the year, month, and day_of_month
Select Cast(year || '-' || month || '-' || day_of_month as date)
From Test_ONTIME_REPORTING
limit 2;

 * postgresql://student@/week6
2 rows affected.


date
2019-01-07
2019-01-07


In [9]:
%%sql
-- here we update the concatenated data into Flight_date column of the test table and check the table
UPDATE Test_ONTIME_REPORTING
set Flight_date = Cast(year || '-' || month || '-' || day_of_month as date);

Select * from Test_ONTIME_REPORTING limit 2;

 * postgresql://student@/week6
20176 rows affected.


[]

In [11]:
%%sql
-- here we check if there is any rows of Flight_date column in the test table not updated
Select * from Test_ONTIME_REPORTING
WHERE Flight_date is null;

 * postgresql://student@/week6
0 rows affected.


report_id,year,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay_new,dep_del15,dep_time_blk,crs_arr_time,arr_time,arr_delay_new,arr_time_blk,cancelled,cancellation_code,crs_elapsed_time,actual_elapsed_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,x,flight_date


In [12]:
%%sql
-- here we delete the year, month, and day_of_month; then check the table
Alter table Test_ONTIME_REPORTING drop column year; 
Alter table Test_ONTIME_REPORTING drop column month;
Alter table Test_ONTIME_REPORTING drop column day_of_month;

Select * from Test_ONTIME_REPORTING limit 2;
-- since our test table looks good, we decide to work on the real ONTIME_REPORTING table

 * postgresql://student@/week6
Done.
Done.
Done.
2 rows affected.


report_id,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay_new,dep_del15,dep_time_blk,crs_arr_time,arr_time,arr_delay_new,arr_time_blk,cancelled,cancellation_code,crs_elapsed_time,actual_elapsed_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,x,flight_date
9E3354123232019325N836AY,1,9E,N836AY,3354,12323,ILM,"Wilmington, NC",10397,ATL,"Atlanta, GA",1841,1834,0,0,1800-1859,2019,123,None,2000-2059,0,None,98,None,377,2,None,None,None,None,None,None,2019-03-25
9E3414129532019318N272PQ,1,9E,N272PQ,3414,12953,LGA,"New York, NY",10792,BUF,"Buffalo, NY",2150,2140,0,0,2100-2159,2316,2252,0,2300-2359,0,None,86,72,292,2,None,None,None,None,None,None,2019-03-18


<font color='green'>Working on the actual ONTIME_REPORTING</font>

In [62]:
%%sql
ALTER table ONTIME_REPORTING 
ADD COLUMN Flight_date DATE NULL;

 * postgresql://student@/week6
Done.


[]

In [63]:
%%sql
Select * from ONTIME_REPORTING limit 2;

 * postgresql://student@/week6
2 rows affected.


report_id,year,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay_new,dep_del15,dep_time_blk,crs_arr_time,arr_time,arr_delay_new,arr_time_blk,cancelled,cancellation_code,crs_elapsed_time,actual_elapsed_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,x,flight_date
9E328010397201916N8694A,2019,1,6,7,9E,N8694A,3280,10397,ATL,"Atlanta, GA",11150,CSG,"Columbus, GA",1645,1643,0,0,1600-1659,1732,1720,0,1700-1759,0,None,47,37,83,1,None,None,None,None,None,None,None
9E328010397201917N8970D,2019,1,7,1,9E,N8970D,3280,10397,ATL,"Atlanta, GA",11150,CSG,"Columbus, GA",1645,1640,0,0,1600-1659,1732,1712,0,1700-1759,0,None,47,32,83,1,None,None,None,None,None,None,None


Now we have the column flight_date added, we can start input value.

In [64]:
%%sql
UPDATE ONTIME_REPORTING
set Flight_date = Cast(year || '-' || month || '-' || day_of_month as date);

 * postgresql://student@/week6
3579077 rows affected.


[]

In [65]:
%%sql
Select * from ONTIME_REPORTING limit 2;

 * postgresql://student@/week6
2 rows affected.


report_id,year,month,day_of_month,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay_new,dep_del15,dep_time_blk,crs_arr_time,arr_time,arr_delay_new,arr_time_blk,cancelled,cancellation_code,crs_elapsed_time,actual_elapsed_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,x,flight_date
9E3342145762019130N8970D,2019,1,30,3,9E,N8970D,3342,14576,ROC,"Rochester, NY",12953,LGA,"New York, NY",1436,None,None,None,1400-1459,1600,None,None,1600-1659,1,B,84,None,254,2,None,None,None,None,None,None,2019-01-30
9E3371145762019120N195PQ,2019,1,20,7,9E,N195PQ,3371,14576,ROC,"Rochester, NY",12478,JFK,"New York, NY",520,None,None,None,0001-0559,700,None,None,0700-0759,1,B,100,None,264,2,None,None,None,None,None,None,2019-01-20


Now our data is filled successfully. We can drop year, month, and day_of_month columns.

In [66]:
%%sql
Alter table ONTIME_REPORTING drop column year; 
Alter table ONTIME_REPORTING drop column month;
Alter table ONTIME_REPORTING drop column day_of_month;

Select * from ONTIME_REPORTING limit 2;
-- now we successfully get rid of year, month, and day_of_month columns

 * postgresql://student@/week6
Done.
Done.
Done.
2 rows affected.
(psycopg2.ProgrammingError) can't execute an empty query
[SQL: -- now we successfully get rid of year, month, and day_of_month columns]
(Background on this error at: https://sqlalche.me/e/14/f405)


#### <font color='bronze'>4. Data analysis (20 points)</font>

##### <font color='grey'> 1) Top 10 airlines in terms of number of scheduled flights. On each of the top 10 airlines show total flights, the sum of all delays by type, and total cancellations. </font>

In [67]:
%%sql
-- AIRLINE is carrier_name
-- Sum of all delays by type (in minutes)
SELECT C.carrier_name as "airline", 
    count(*) as "total scheduled flights", 
    sum(OP.carrier_delay) as "sum of carrier delay",
    sum(OP.weather_delay) as "sum of weather delay",
    sum(OP.nas_delay) as "sum of nas delay",
    sum(OP.security_delay) as "sum of security delay",
    sum(OP.late_aircraft_delay) as "sum of late aircraft delay",
    sum(OP.cancelled) as "total cancellations"
from ONTIME_REPORTING OP, CARRIER C
WHERE C.op_unique_carrier = OP.op_unique_carrier
GROUP BY C.carrier_name
ORDER BY count(*) Desc
LIMIT 10

 * postgresql://student@/week6
10 rows affected.


airline,total scheduled flights,sum of carrier delay,sum of weather delay,sum of nas delay,sum of security delay,sum of late aircraft delay,total cancellations
Southwest Airlines Co.,658265,1509844,74316,705725,19991,1759119,36722
Delta Air Lines Inc.,467185,1327495,325429,965721,3363,1066182,20174
American Airlines Inc.,457111,1758590,181434,1039563,7618,1799175,20782
SkyWest Airlines Inc.,407303,2529666,698177,1131810,4781,1634730,17457
United Air Lines Inc.,292736,729115,157040,1098214,546,1283961,14297
"Midwest Airline, Inc.",163469,362907,75630,540814,1059,613239,7066
American Eagle Airlines Inc.,153185,423455,135555,445035,1148,721184,8670
JetBlue Airways,146661,806009,40894,447768,3124,844491,6215
Comair Inc.,141506,531257,64036,245468,2009,955513,7076
Endeavor Air Inc.,131567,387508,103785,324399,291,601609,4754


ANS:
top 10 airlines in terms of number of scheduled flights are:
1. Southwest Airlines Co.
2. Delta Air Lines Inc.
3. American Airlines Inc.
4. SkyWest Airlines Inc.
5. United Air Lines Inc.
6. Midwest Airline, Inc.
7. American Eagle Airlines Inc.
8. JetBlue Airways
9. Comair Inc.
10. Endeavor Air Inc.

In [14]:
%%sql
SELECT distinct cancelled from ONTIME_REPORTING
-- cancelled is binary so that we can use sum() function to count total cancellations

 * postgresql://student@/week6
2 rows affected.


cancelled
1
0


%%sql
-- AIRLINE is carrier_name </br>
-- This code is used to count how many times of delays happen by type </br>
SELECT C.carrier_name as "airline", 
    count(*) as "total scheduled flights", 
    count(OP.carrier_delay) - sum(CASE WHEN OP.carrier_delay = 0 THEN 1 ELSE 0 END) as "total carrier delay",
    count(OP.weather_delay) - sum(CASE WHEN OP.weather_delay = 0 THEN 1 ELSE 0 END) as "total weather delay",
    count(OP.nas_delay) - sum(CASE WHEN OP.nas_delay = 0 THEN 1 ELSE 0 END) as "total nas delay",
    count(OP.security_delay) - sum(CASE WHEN OP.security_delay = 0 THEN 1 ELSE 0 END) as "total security delay",
    count(OP.late_aircraft_delay) - sum(CASE WHEN OP.late_aircraft_delay = 0 THEN 1 ELSE 0 END) as "total late aircraft delay",
    sum(OP.cancelled) as "total cancellations"
from ONTIME_REPORTING OP, CARRIER C
WHERE C.op_unique_carrier = OP.op_unique_carrier
GROUP BY C.carrier_name
ORDER BY count(*) Desc
LIMIT 10

##### <font color='grey'> 2) Top 10 busiest airport pairs in terms of total scheduled flights? Of those, indicate the airlines, total number of seats they can offer. </font>

In [68]:
%%sql
SELECT AO.DISPLAY_AIRPORT_NAME || '-' || AD.DISPLAY_AIRPORT_NAME as "busiest airport pairs", 
    C.carrier_name as "airline",
    count(*) as "total scheduled flights",
    SUM(A.NUMBER_OF_SEATS) as  "total number of seats"
FROM   ONTIME_REPORTING OP, AIRCRAFT A, CARRIER C, AIRPORT_LIST AO, AIRPORT_LIST AD
WHERE  A.tail_num = OP.tail_num 
    and C.op_unique_carrier = OP.op_unique_carrier
    and OP.origin_airport_id = AO.origin_airport_id
    and OP.dest_airport_id=AD.origin_airport_id
group by AO.DISPLAY_AIRPORT_NAME || '-' || AD.DISPLAY_AIRPORT_NAME, C.carrier_name
ORDER BY count(*) Desc
limit 10

 * postgresql://student@/week6
10 rows affected.


busiest airport pairs,airline,total scheduled flights,total number of seats
Hollywood-Burbank Midpoint-Kahului Airport,Hawaiian Airlines Inc.,4680,578478
Kahului Airport-Hollywood-Burbank Midpoint,Hawaiian Airlines Inc.,4673,577617
William P Hobby-Dallas Love Field,Southwest Airlines Co.,3320,487752
Dallas Love Field-William P Hobby,Southwest Airlines Co.,3300,482940
Hollywood-Burbank Midpoint-Lihue Airport,Hawaiian Airlines Inc.,3131,385113
Lihue Airport-Hollywood-Burbank Midpoint,Hawaiian Airlines Inc.,3123,384129
Hollywood-Burbank Midpoint-Keahole,Hawaiian Airlines Inc.,3011,370353
Keahole-Hollywood-Burbank Midpoint,Hawaiian Airlines Inc.,3007,369861
Orlando International-Atlanta Municipal,Delta Air Lines Inc.,2900,582744
Atlanta Municipal-Orlando International,Delta Air Lines Inc.,2894,581626


ANS:
The top10 busiest airport pairs in terms of total scheduled flights are:
 1. Hollywood-Burbank Midpoint-Kahului Airport Hawaiian Airlines Inc.
 2. Kahului Airport-Hollywood-Burbank Midpoint Hawaiian Airlines Inc.
 3. William P Hobby-Dallas Love Field Southwest Airlines Co.
 4. Dallas Love Field-William P Hobby Southwest Airlines Co.
 5. Hollywood-Burbank Midpoint-Lihue Airport Hawaiian Airlines Inc.
 6. Lihue Airport-Hollywood-Burbank Midpoint Hawaiian Airlines Inc.
 7. Hollywood-Burbank Midpoint-Keahole Hawaiian Airlines Inc.
 8. Keahole-Hollywood-Burbank Midpoint Hawaiian Airlines Inc.
 9. Orlando International-Atlanta Municipal Delta Air Lines Inc.
 10. Atlanta Municipal-Orlando International Delta Air Lines Inc.

##### <font color='grey'> 3) Top 10 snow days (SNOW field). For each find the corresponding date and airport. Did any flight departed or landed from those airports on those days? </font>

In [69]:
%%sql
WITH
T10SNOW_D as (
    SELECT AL.display_airport_name, W.date, W.snow, count(*) - sum(cancelled) as "flight_departed"
    FROM WEATHER W LEFT OUTER JOIN AIRPORT_LIST AL ON (LOWER(AL.name) = LOWER(W.name)), ONTIME_REPORTING OP
    WHERE W.date = OP.flight_date and OP.origin_airport_id = AL.origin_airport_id
    GROUP BY AL.display_airport_name, W.date, W.snow 
    ORDER BY W.snow Desc NULLS LAST
    LIMIT 10),
T10SNOW_L as (
    SELECT AL.display_airport_name, W.date, W.snow, count(*) - sum(cancelled) as "flight_landed"
    FROM WEATHER W LEFT OUTER JOIN AIRPORT_LIST AL ON (LOWER(AL.name) = LOWER(W.name)), ONTIME_REPORTING OP
    WHERE W.date = OP.flight_date and OP.dest_airport_id = AL.origin_airport_id
    GROUP BY AL.display_airport_name, W.date, W.snow 
    ORDER BY W.snow Desc NULLS LAST
    LIMIT 10)
SELECT D.display_airport_name, D.date, D.snow, D.flight_departed, L.flight_landed
FROM T10SNOW_D D
LEFT JOIN T10SNOW_L L ON (D.display_airport_name = L.display_airport_name) and (D.date = L.date)
ORDER BY D.snow Desc;

 * postgresql://student@/week6
10 rows affected.


display_airport_name,date,snow,flight_departed,flight_landed
Greater Buffalo International,2019-01-25,17.20,61,60
Greater Buffalo International,2019-01-30,13.60,2,2
Albany International,2019-01-20,10.40,1,3
Logan International,2019-03-04,9.90,255,298
Syracuse Hancock International,2019-01-20,9.30,13,20
Syracuse Hancock International,2020-02-07,9.20,26,26
Minneapolis-St Paul International,2019-02-20,9.10,18,21
Anchorage International,2020-02-18,8.90,39,41
Salt Lake City International,2020-02-03,8.60,336,335
General Mitchell Field,2019-02-12,8.30,61,66


ANS:
    Top 10 snow days( airport and date):
1. Greater Buffalo International     2019-01-25
2. Greater Buffalo International     2019-01-30
3. Albany International              2019-01-20
4. Logan International               2019-03-04
5. Syracuse Hancock International    2019-01-20
6. Syracuse Hancock International    2020-02-07
7. Minneapolis-St Paul International 2019-02-20
8. Anchorage International           2020-02-18
9. Salt Lake City International      2020-02-03
10.General Mitchell Field            2019-02-12

We use WITH to generate 2 tables and combine them to display aircraft departed and landed. All the airports of the top 10 snow have flight departed and landed in the certain day. The accurate number of flight departed and landed showed above in the table.

<font color='green'>Explanation</font>

We test our result by running a query on WEATHER table only. </br> 
We find out that 'BURLINGTON INTERNATIONAL AIRPORT, VT US' is not on our departed-lander-of-top-10-snow-day query while is presented on top-10-snow-day query. </br> We further investigate by looking for the 'BURLINGTON INTERNATIONAL AIRPORT, VT US' in AIRPORT_LIST table. By doing so, we find out that it is not on departed-lander-of-top-10-snow-day query because it is not available in AIRPORT_LIST AL table and thus the full name (WEATHER.name) cannot match to the ONTIME_REPORTING OP(origin_airport_id/dest_airport_id) through AIRPORT_LIST table (AL.name = WEATHER.name and OP.origin_airport_id/OP.origin_airport_id = AL.origin_airport_id). </br> 
Therefore, our query gives the corresponding top 11 snow days with the omission of 'BURLINGTON INTERNATIONAL AIRPORT, VT US' which eventually affect 10 rows.

In [60]:
%%sql
SELECT * FROM WEATHER ORDER BY snow Desc NULLS LAST LIMIT 11

 * postgresql://student@/week6
11 rows affected.


s_date,station,name,date,prcp,snow,snwd,tmax,awnd
USW0001473320190125,USW00014733,"BUFFALO NIAGARA INTERNATIONAL, NY US",2019-01-25,0.87,17.20,5.90,28.00,18.57
USW0001474220190120,USW00014742,"BURLINGTON INTERNATIONAL AIRPORT, VT US",2019-01-20,0.50,15.60,18.10,5.00,10.96
USW0001473320190130,USW00014733,"BUFFALO NIAGARA INTERNATIONAL, NY US",2019-01-30,0.89,13.60,18.90,11.00,17.67
USW0001473520190120,USW00014735,"ALBANY INTERNATIONAL AIRPORT, NY US",2019-01-20,1.25,10.40,11.00,21.00,10.96
USW0001473920190304,USW00014739,"BOSTON, MA US",2019-03-04,0.97,9.90,None,40.00,13.20
USW0001477120190120,USW00014771,"SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US",2019-01-20,0.57,9.30,15.00,19.00,11.86
USW0001477120200207,USW00014771,"SYRACUSE HANCOCK INTERNATIONAL AIRPORT, NY US",2020-02-07,0.90,9.20,5.10,33.00,11.18
USW0001492220190220,USW00014922,"MINNEAPOLIS ST. PAUL INTERNATIONAL AIRPORT, MN US",2019-02-20,0.60,9.10,11.00,28.00,10.74
USW0002645120200218,USW00026451,"ANCHORAGE TED STEVENS INTERNATIONAL AIRPORT, AK US",2020-02-18,0.88,8.90,11.80,44.00,10.51
USW0002412720200203,USW00024127,"SALT LAKE CITY INTERNATIONAL AIRPORT, UT US",2020-02-03,0.41,8.60,5.10,29.00,16.33


In [56]:
%%sql
SELECT *
FROM AIRPORT_LIST AL
WHERE AL.name = 'BURLINGTON INTERNATIONAL AIRPORT, VT US'

 * postgresql://student@/week6
0 rows affected.


origin_airport_id,display_airport_name,origin_city_name,name


Test on our output to see if our count is accurate: this test match our output which says 'Albany International' have 1 departed and 3 landed on 2019-01-20

In [21]:
%%sql
SELECT *
FROM AIRPORT_LIST AL
WHERE AL.display_airport_name = 'Albany International'

 * postgresql://student@/week6
1 rows affected.


origin_airport_id,display_airport_name,origin_city_name,name
10257,Albany International,"Albany, NY","ALBANY INTERNATIONAL AIRPORT, NY US"


In [23]:
%%sql
SELECT *
FROM ONTIME_REPORTING OP
WHERE (origin_airport_id = 10257 or dest_airport_id = 10257) and flight_date = '2019-01-20' and cancelled = 0

 * postgresql://student@/week6
4 rows affected.


report_id,day_of_week,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay_new,dep_del15,dep_time_blk,crs_arr_time,arr_time,arr_delay_new,arr_time_blk,cancelled,cancellation_code,crs_elapsed_time,actual_elapsed_time,distance,distance_group,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,x,flight_date
DL2228103972019120N979DL,7,DL,N979DL,2228,10397,ATL,"Atlanta, GA",10257,ALB,"Albany, NY",1952,2050,58,1,1900-1959,2207,2310,63,2200-2259,0,None,135,140,853,4,0,0,63,0,0,None,2019-01-20
AA1905110572019120N717UW,7,AA,N717UW,1905,11057,CLT,"Charlotte, NC",10257,ALB,"Albany, NY",1630,1629,0,0,1600-1659,1829,None,None,1800-1859,0,None,119,None,646,3,None,None,None,None,None,None,2019-01-20
DL1875102572019120N912DE,7,DL,N912DE,1875,10257,ALB,"Albany, NY",10397,ATL,"Atlanta, GA",1400,957,1197,1,1400-1459,1706,1347,1241,1700-1759,0,None,186,230,853,4,0,1198,43,0,0,None,2019-01-20
DL1875103972019120N912DE,7,DL,N912DE,1875,10397,ATL,"Atlanta, GA",10257,ALB,"Albany, NY",1031,1718,407,1,1000-1059,1248,1956,428,1200-1259,0,None,137,158,853,4,0,0,428,0,0,None,2019-01-20


Breaking down the code to departed flight and landed flight 

In [11]:
%%sql
-- left out join is used to display even if departed and landed is None
SELECT AL.display_airport_name, W.date, W.snow, count(*) - sum(cancelled) as "flight departed"
FROM WEATHER W LEFT OUTER JOIN AIRPORT_LIST AL ON (LOWER(AL.name) = LOWER(W.name)), ONTIME_REPORTING OP
WHERE W.date = OP.flight_date and 
    OP.origin_airport_id = AL.origin_airport_id
GROUP BY AL.display_airport_name, W.date, W.snow 
ORDER BY W.snow Desc NULLS LAST
LIMIT 10;

 * postgresql://student@/week6
10 rows affected.


display_airport_name,date,snow,flight departed
Greater Buffalo International,2019-01-25,17.20,61
Greater Buffalo International,2019-01-30,13.60,2
Albany International,2019-01-20,10.40,1
Logan International,2019-03-04,9.90,255
Syracuse Hancock International,2019-01-20,9.30,13
Syracuse Hancock International,2020-02-07,9.20,26
Minneapolis-St Paul International,2019-02-20,9.10,18
Anchorage International,2020-02-18,8.90,39
Salt Lake City International,2020-02-03,8.60,336
General Mitchell Field,2019-02-12,8.30,61


In [16]:
%%sql
-- left out join is used to display even if departed and landed is None
SELECT AL.display_airport_name, W.date, W.snow, count(*) - sum(cancelled) as "flight landed"
FROM WEATHER W LEFT OUTER JOIN AIRPORT_LIST AL ON (LOWER(AL.name) = LOWER(W.name)), ONTIME_REPORTING OP
WHERE W.date = OP.flight_date and 
    OP.dest_airport_id = AL.origin_airport_id
GROUP BY AL.display_airport_name, W.date, W.snow 
ORDER BY W.snow Desc NULLS LAST
LIMIT 10;

 * postgresql://student@/week6
10 rows affected.


display_airport_name,date,snow,flight landed
Greater Buffalo International,2019-01-25,17.20,60
Greater Buffalo International,2019-01-30,13.60,2
Albany International,2019-01-20,10.40,3
Logan International,2019-03-04,9.90,298
Syracuse Hancock International,2019-01-20,9.30,20
Syracuse Hancock International,2020-02-07,9.20,26
Minneapolis-St Paul International,2019-02-20,9.10,21
Anchorage International,2020-02-18,8.90,41
Salt Lake City International,2020-02-03,8.60,335
General Mitchell Field,2019-02-12,8.30,66
